# fNIRS Preprocessing pipeline & conversion to GAF-images

This document includes the code for the preprocessing of the collected fNIRS data using the MNE-nirs toolbox as well as the code for converting the segmented data into GAF images


## Importing the relevant packages and libraries

In [1]:
# Importing packages and libraries related to the MNE-nirs toolbox
import mne as mne
import mne_nirs
from mne.preprocessing.nirs import beer_lambert_law, optical_density
from mne.io import read_raw_snirf
import mne_bids
from mne.preprocessing.nirs import (optical_density,
                                    temporal_derivative_distribution_repair)

# Additional packages 
import pyvista
import pyvistaqt
import pandas as pd
import numba
import numpy as np
from itertools import compress
import os

# Packages related to image conversion
import matplotlib.pyplot as plt
from PIL import Image as im
from matplotlib import cm
from pyts.image import GramianAngularField


/Users/klarafomsgaard/opt/anaconda3/lib/python3.9/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "
/Users/klarafomsgaard/opt/anaconda3/lib/python3.9/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '


## Preprocessing pipeline for fNIRS using MNE
The pipeline is written as a function as this allows for looping over all participants, applying the same pipeline to all.

In [2]:

def preproc_mne(filepath):
    
    # -- LOADING IN THE DATA --
    # Set pyvista to be the 3D backend
    mne.viz.set_3d_backend("pyvista") 

    # Find the .snirf file containing the fNIRS data for the participant
    fname = filepath 

    # Getting the raw intensity 
    raw_intensity = mne.io.read_raw_snirf(fname, verbose = True, preload = True).load_data()
    
    # raw optical density
    raw_od = mne.preprocessing.nirs.optical_density(raw_intensity)

    


    # -- INVESTIGATION OF SCALP COUPLING INDEX --

    # sci = mne.preprocessing.nirs.scalp_coupling_index(raw_od)
    # raw_od.info['bads'] = list(compress(raw_od.ch_names, sci < 0.5))
    # print(raw_od.info['bads'])
    # fig,ax=plt.subplots()
    # ax.hist(sci)
    # ax.set(xlabel='Scalp Coupling Index', ylabel='Count',xlim=[0,1])

    ## This is not active code, at it was used in an earlier stage to find bad channels (sci < 0.5) for all participants
    ## All bad channels were noted and consistently bad channels across participants were filtered out using the following code:

    raw_od.info['bads'].extend(["S3_D4 760","S3_D4 850","S8_D3 760","S8_D3 850","S8_D7 760","S8_D7 850",
    "S11_D9 760","S11_D9 850","S11_D11 760","S11_D11 850","S13_D14 760","S13_D14 850","S13_D37 760","S13_D37 850",
    "S15_D14 760","S15_D14 850","S15_D15 760","S15_D15 850","S17_D19 760","S17_D19 850","S21_D18 760","S21_D18 850",
    "S25_D22 760","S25_D22 850","S25_D24 760","S25_D24 850","S26_D23 760","S26_D23 850","S26_D24 760","S26_D24 850","S26_D25 760",
    "S26_D25 850","S26_D26 760","S26_D26 850","S28_D27 760","S28_D27 850","S28_D29 760","S28_D29 850","S32_D29 760","S32_D29 850"])

    bads_list = raw_od.info["bads"]
    raw_od_no_bad = raw_od.copy().drop_channels(bads_list)



    # -- ARTIFACT DETECTION & CORRECTION --

    # Short channel motion correction
    od_corrected = mne_nirs.signal_enhancement.short_channel_regression(raw_od_no_bad)
    
    # Motion correction and baseline shift using TDDR
    corrected_tddr = temporal_derivative_distribution_repair(od_corrected)



    # -- CONVERSION TO HBO & HBR 

    # Converting to change in oxy- and deoxyhemoglobin levels using the modified Lambert Beer Law
    raw_hemo = beer_lambert_law(corrected_tddr,ppf=0.1)

    # Enhancement of negative correlation between HbO and HbR
    raw_hemo = mne_nirs.signal_enhancement.enhance_negative_correlation(raw_hemo)

    # Only include the long channels, as the short channels are for motion artifact detection
    raw_hemo_long = mne_nirs.channels.get_long_channels(raw_hemo)
 
    # Apply bandpass filter to data
    raw_hemo_long = raw_hemo_long.filter(0.01,0.7,h_trans_bandwidth = 0.3, l_trans_bandwidth=0.005) 
 

    # -- INSPECTION OF CLEANED DATA --
    plt.rcParams["figure.figsize"]=(16,10)
    raw_hemo_long.plot(n_channels=50, duration=100, show_scrollbars=False)

    # -- DATA SEGMENTATION --
    # Generate events for epoching 
    events,event_id = mne.events_from_annotations(raw_hemo_long)
    
    # Epoching the data (segmentation into trial relevant sections)
    reject_criteria = dict(hbo=80e-5) # Set rejection criterion
    tmin,tmax = -2,7 # Timeframe for epochs: 2 second before and 7 seconds after event onset
    epochs = mne.Epochs(raw_hemo_long,events,event_id = event_id,
                        tmin=tmin,tmax=tmax,preload=True, baseline = (None,0), 
                        reject=reject_criteria, reject_by_annotation=True)

    return epochs



## Image conversion to GAF images


#### Function used to create the filename for the images

In [ ]:
# Make function to append suffixes
def append_suffix(nr,channels, filename, id_phase, epochnr, stim):
    name, ext = os.path.splitext(filename)
    return "{nr}-{channels}_{id_phase}_{epochnr}_{stim}{ext}".format(nr = nr, channels=channels, id_phase = id_phase, stim=stim, epochnr = epochnr, ext=ext)

#### Functions to create folders 
The function together create one folder for each participant where images for each epoch is saved in separate folders

In [ ]:
# Required package
from os import mkdir

def createfolder_part(participant):
    dir_part = os.path.join("/path/to/where/we/want/to/save/the/images",f"{participant}")
    if not os.path.exists(dir_part):
        os.mkdir(dir_part)
    return dir_part

def createfolder_epoch(dir_part,epochnr,stim):
    dir_epoch = os.path.join(f"{dir_part}",f"{epochnr}_{stim}")
    if not os.path.exists(dir_epoch):
        os.mkdir(dir_epoch)
    return dir_epoch

#### Function that converts the data to a GAF image per channel per epoch

The function converts haemodynamic response signals for each epoch into a 2D GAF image with the dimensions 36x36 pixels (timepoints x timepoints). 


Input:

*epoch_basestim*: The epoched and preprocessed heamodynamic response signals extracted from the raw fNIRS data

*id_phase*: The participant id and phase - this is used to include in the filename of the image

*hbo*: Decides whether the images should include hbr channels or only hbo

In [3]:

def norm_channel(epoch_basestim,id_phase,hbo):

    # Create a folder for the participant
    dir_part = createfolder_part(id_phase)

    # -- CONVERT DATA INTO DATAFRAME --
    if hbo == True: # This is run if only the hbo channels are desired
        df_channels_base = epoch_basestim.copy().pick("hbo")
        df_channels = df_channels_base.to_data_frame()

    else: # Run if we want both the hbo and hbr channels
        df_channels_base_hbo = epoch_basestim.copy().pick("hbo") 
        df_channels_base_hbo_df = df_channels_base_hbo.to_data_frame()

        df_channels_base_hbr = epoch_basestim.copy().pick("hbr") 
        df_channels_base_hbr_df = df_channels_base_hbr.to_data_frame()

        df_channels = df_channels_base_hbo_df.join(df_channels_base_hbr_df.iloc[:,3:])


    # -- LOOPING OVER EACH EPOCH AND CONVERTING THE DATA FOR EACH CHANNEL TO GAF --
    for epoch in np.nditer(df_channels['epoch'].unique()):
        epochnr = epoch 
        
        # Selecting only one epoch and prepare for image conversion
        epochsubset = df_channels.loc[df_channels['epoch'] == epoch] 
        stim = epochsubset.iloc[1]['condition'] # Save the condition to use in the filename when saving the image 
        epoch_df_clean = epochsubset.drop(["time","condition","epoch"],axis=1).transpose() # Clean the dataframe and arrange to one column per channel
        epoch_array = epoch_df_clean.to_numpy() 

        # Convert data from each channel into a GAF image
        gaf = GramianAngularField()
        image = gaf.transform(epoch_array)
        plt.set_cmap('rainbow')
        filename = "epochs.png" 
        nr = 1
        for i in image:  
            if hbo == True: # Save the images in different folders depending on whether they include hbr channels or not
                fullname = append_suffix(nr,"hbo",filename,id_phase,epochnr,stim)
                image_path = createfolder_epoch(dir_part,epochnr,stim)
            else: 
                fullname = append_suffix(nr,"all",filename,id_phase,epochnr,stim)
                image_path = createfolder_epoch(dir_part,epochnr,stim)
            plt.imsave(f"{image_path}/{fullname}",i) 
            nr = nr+1
        print(f"Epoch{epochnr}, channel {i} saved")


#### Creating the images including only HbO channels

In [ ]:
# Making a list of participants
dir_path = "/path/to/.snirf-files"
import glob 
snirf_list = glob.glob(f"{dir_path}/*")
if '.DS_Store' in snirf_list:
    snirf_list.remove('.DS_Store')

# Loop over all participants using the above created functions
for i in snirf_list:
    print(f"loading {i}")
    epochs = preproc_mne(f"{i}")
    id_phase = i.split("/")[6].split(".")[0] # This needs adjusting depending on your own path
    norm_channel(epochs,id_phase,True)
